In [1]:
import pandas as pd
import sqlite3

# Load cleaned CSV
df = pd.read_csv(r'C:\Users\bheemanathi\Downloads\ecommerce_cleaned.csv')

# Create SQLite database
conn = sqlite3.connect('ecommerce.db')

# Load dataframe into SQL table
df.to_sql('ecommerce', conn, if_exists='replace', index=False)

print(f"✅ Database created!")
print(f"✅ Table 'ecommerce' loaded with {len(df):,} rows")

✅ Database created!
✅ Table 'ecommerce' loaded with 100,000 rows


In [2]:
# Helper function — run any SQL query easily
def run_query(sql):
    return pd.read_sql_query(sql, conn)

# Test
result = run_query("SELECT COUNT(*) as total_rows FROM ecommerce")
print(result)
print("✅ SQL is working!")

   total_rows
0      100000
✅ SQL is working!


## Total Revenue, Profit, ROAS

In [3]:
run_query("""
    SELECT 
        ROUND(SUM(Revenue), 2)        AS Total_Revenue,
        ROUND(SUM(Profit), 2)         AS Total_Profit,
        ROUND(SUM(Ad_Spend), 2)       AS Total_Ad_Spend,
        ROUND(AVG(ROAS), 2)           AS Avg_ROAS,
        COUNT(Transaction_ID)         AS Total_Orders,
        ROUND(AVG(Revenue), 2)        AS Avg_Order_Value
    FROM ecommerce
""")

,Total_Revenue,Total_Profit,Total_Ad_Spend,Avg_ROAS,Total_Orders,Avg_Order_Value
0,55311081.07,44263357.1,11047723.97,13.66,100000,553.11


## Monthly Revenue Trend

In [4]:
run_query("""
    SELECT 
        YearMonth,
        Month_Name,
        COUNT(Transaction_ID)     AS Total_Orders,
        ROUND(SUM(Revenue), 2)    AS Monthly_Revenue,
        ROUND(SUM(Profit), 2)     AS Monthly_Profit,
        ROUND(AVG(Revenue), 2)    AS Avg_Order_Value
    FROM ecommerce
    GROUP BY YearMonth, Month_Name
    ORDER BY YearMonth
""")

,YearMonth,Month_Name,Total_Orders,Monthly_Revenue,Monthly_Profit,Avg_Order_Value
0,2023-12,December,6814,3723069.77,2978616.76,546.39
1,2024-01,January,8444,4665609.02,3728434.03,552.54
2,2024-02,February,7831,4312157.04,3448075.94,550.65
3,2024-03,March,8588,4705422.31,3746957.14,547.91
4,2024-04,April,8041,4406829.73,3521136.23,548.04
5,2024-05,May,8465,4645304.43,3713384.19,548.77
6,2024-06,June,8300,4614620.94,3694142.98,555.98
7,2024-07,July,8419,4572138.90,3650113.23,543.07
8,2024-08,August,8324,4579761.33,3656963.97,550.19
9,2024-09,September,8281,4551504.34,3639047.70,549.63


## Revenue by Category

In [5]:
run_query("""
    SELECT 
        Category,
        COUNT(Transaction_ID)          AS Total_Orders,
        ROUND(SUM(Revenue), 2)         AS Total_Revenue,
        ROUND(SUM(Profit), 2)          AS Total_Profit,
        ROUND(SUM(Ad_Spend), 2)        AS Total_Ad_Spend,
        ROUND(AVG(Profit_Margin_Pct), 2) AS Avg_Profit_Margin,
        ROUND(AVG(ROAS), 2)            AS Avg_ROAS
    FROM ecommerce
    GROUP BY Category
    ORDER BY Total_Revenue DESC
""")

,Category,Total_Orders,Total_Revenue,Total_Profit,Total_Ad_Spend,Avg_Profit_Margin,Avg_ROAS
0,Electronics,21533,11326820.25,8944387.67,2382432.58,45.92,12.72
1,Books,19006,11195311.88,9085441.30,2109870.58,48.84,14.57
2,Toys,20764,11194842.46,8906124.20,2288718.26,48.54,13.28
3,Clothing,19278,11034899.26,8911514.93,2123384.33,54.39,14.13
4,Home Appliances,19419,10559207.22,8415889.00,2143318.22,36.49,13.76


## Revenue by Region

In [6]:
run_query("""
    SELECT 
        Region,
        COUNT(Transaction_ID)       AS Total_Orders,
        ROUND(SUM(Revenue), 2)      AS Total_Revenue,
        ROUND(SUM(Profit), 2)       AS Total_Profit,
        ROUND(AVG(ROAS), 2)         AS Avg_ROAS
    FROM ecommerce
    GROUP BY Region
    ORDER BY Total_Revenue DESC
""")

,Region,Total_Orders,Total_Revenue,Total_Profit,Avg_ROAS
0,Asia,33472,18492871.13,14793767.34,13.75
1,North America,33263,18419228.61,14731567.22,13.66
2,Europe,33265,18398981.33,14738022.54,13.57


## Top 10 Products by Revenue


In [7]:
run_query("""
    SELECT 
        Product_ID,
        COUNT(Transaction_ID)     AS Total_Orders,
        ROUND(SUM(Revenue), 2)    AS Total_Revenue,
        ROUND(SUM(Profit), 2)     AS Total_Profit,
        ROUND(SUM(Units_Sold), 0) AS Total_Units
    FROM ecommerce
    GROUP BY Product_ID
    ORDER BY Total_Revenue DESC
    LIMIT 10
""")

,Product_ID,Total_Orders,Total_Revenue,Total_Profit,Total_Units
0,Product_108,117,125682.19,113237.49,15849.0
1,Product_404,119,125115.04,111570.88,13930.0
2,Product_370,117,123196.60,110123.98,15338.0
3,Product_686,113,121787.93,109481.88,13338.0
4,Product_77,112,121302.64,108734.29,13610.0
5,Product_411,117,121283.88,108525.16,16323.0
6,Product_258,118,121225.16,109532.35,15216.0
7,Product_998,116,120448.50,107469.99,15822.0
8,Product_690,112,120414.76,107974.99,14063.0
9,Product_763,111,120157.60,106587.18,13125.0


## Unique vs Repeat Customers


In [8]:
run_query("""
    SELECT
        CASE WHEN order_count = 1 THEN 'New Customer'
             ELSE 'Repeat Customer' END AS Customer_Type,
        COUNT(*) AS Customer_Count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) AS Percentage
    FROM (
        SELECT Customer_ID, COUNT(Transaction_ID) AS order_count
        FROM ecommerce
        GROUP BY Customer_ID
    )
    GROUP BY Customer_Type
""")

,Customer_Type,Customer_Count,Percentage
0,New Customer,651,3.28
1,Repeat Customer,19214,96.72


## Top 10 Customers by Revenue

In [9]:
run_query("""
    SELECT 
        Customer_ID,
        COUNT(Transaction_ID)     AS Total_Orders,
        ROUND(SUM(Revenue), 2)    AS Total_Revenue,
        ROUND(SUM(Profit), 2)     AS Total_Profit,
        ROUND(AVG(Revenue), 2)    AS Avg_Order_Value
    FROM ecommerce
    GROUP BY Customer_ID
    ORDER BY Total_Revenue DESC
    LIMIT 10
""")

,Customer_ID,Total_Orders,Total_Revenue,Total_Profit,Avg_Order_Value
0,Customer_2379,13,12021.14,10340.82,924.70
1,Customer_18839,14,11207.58,10220.85,800.54
2,Customer_3173,9,10849.05,9912.99,1205.45
3,Customer_10780,11,10784.89,9358.93,980.44
4,Customer_10503,11,10634.27,9784.36,966.75
5,Customer_11994,13,10523.24,8706.19,809.48
6,Customer_16807,11,10499.72,9413.58,954.52
7,Customer_5945,9,10280.72,9204.84,1142.30
8,Customer_16840,13,10018.70,8054.84,770.67
9,Customer_2776,12,9998.02,8947.14,833.17


## Average Order Value by Region & Category


In [10]:
run_query("""
    SELECT 
        Region,
        Category,
        ROUND(AVG(Revenue), 2)   AS Avg_Order_Value,
        ROUND(SUM(Revenue), 2)   AS Total_Revenue,
        COUNT(Transaction_ID)    AS Total_Orders
    FROM ecommerce
    GROUP BY Region, Category
    ORDER BY Avg_Order_Value DESC
""")

,Region,Category,Avg_Order_Value,Total_Revenue,Total_Orders
0,North America,Books,596.26,3823250.11,6412
1,Asia,Books,586.33,3677433.85,6272
2,Europe,Books,584.41,3694627.92,6322
3,North America,Clothing,575.99,3685779.57,6399
4,Europe,Clothing,571.31,3656389.48,6400
5,Asia,Clothing,569.95,3692730.21,6479
6,Europe,Home Appliances,547.38,3502678.30,6399
7,Asia,Home Appliances,545.69,3568283.18,6539
8,Asia,Toys,539.45,3749743.14,6951
9,Europe,Toys,539.28,3749051.27,6952


## Profit by Category


In [11]:
run_query("""
    SELECT 
        Category,
        ROUND(SUM(Revenue), 2)           AS Total_Revenue,
        ROUND(SUM(Profit), 2)            AS Total_Profit,
        ROUND(AVG(Profit_Margin_Pct), 2) AS Avg_Margin_Pct,
        ROUND(SUM(Ad_Spend), 2)          AS Total_Ad_Spend,
        ROUND(AVG(ROAS), 2)              AS Avg_ROAS
    FROM ecommerce
    GROUP BY Category
    ORDER BY Total_Profit DESC
""")

,Category,Total_Revenue,Total_Profit,Avg_Margin_Pct,Total_Ad_Spend,Avg_ROAS
0,Books,11195311.88,9085441.30,48.84,2109870.58,14.57
1,Electronics,11326820.25,8944387.67,45.92,2382432.58,12.72
2,Clothing,11034899.26,8911514.93,54.39,2123384.33,14.13
3,Toys,11194842.46,8906124.20,48.54,2288718.26,13.28
4,Home Appliances,10559207.22,8415889.00,36.49,2143318.22,13.76


## Discount Impact on Profit


In [12]:
run_query("""
    SELECT 
        CASE 
            WHEN Discount_Applied = 0          THEN 'No Discount'
            WHEN Discount_Applied <= 0.1       THEN 'Low (1-10%)'
            WHEN Discount_Applied <= 0.2       THEN 'Medium (11-20%)'
            WHEN Discount_Applied <= 0.3       THEN 'High (21-30%)'
            ELSE                                    'Very High (30%+)'
        END AS Discount_Range,
        COUNT(Transaction_ID)            AS Total_Orders,
        ROUND(SUM(Revenue), 2)           AS Total_Revenue,
        ROUND(SUM(Profit), 2)            AS Total_Profit,
        ROUND(AVG(Profit_Margin_Pct), 2) AS Avg_Margin_Pct
    FROM ecommerce
    GROUP BY Discount_Range
    ORDER BY Avg_Margin_Pct DESC
""")

,Discount_Range,Total_Orders,Total_Revenue,Total_Profit,Avg_Margin_Pct
0,Low (1-10%),33308,20422934.14,16737418.90,53.01
1,No Discount,1685,1107456.99,920377.61,51.77
2,Medium (11-20%),33431,18399855.80,14718680.60,46.81
3,High (21-30%),31576,15380834.14,11886879.99,40.04


## Monthly Profit Trend


In [13]:
run_query("""
    SELECT
        YearMonth,
        ROUND(SUM(Revenue), 2)           AS Revenue,
        ROUND(SUM(Profit), 2)            AS Profit,
        ROUND(AVG(Profit_Margin_Pct), 2) AS Avg_Margin_Pct,
        ROUND(SUM(Ad_Spend), 2)          AS Ad_Spend
    FROM ecommerce
    GROUP BY YearMonth
    ORDER BY YearMonth
""")

,YearMonth,Revenue,Profit,Avg_Margin_Pct,Ad_Spend
0,2023-12,3723069.77,2978616.76,45.99,744453.01
1,2024-01,4665609.02,3728434.03,47.70,937174.99
2,2024-02,4312157.04,3448075.94,49.36,864081.10
3,2024-03,4705422.31,3746957.14,43.71,958465.17
4,2024-04,4406829.73,3521136.23,47.76,885693.50
5,2024-05,4645304.43,3713384.19,46.73,931920.24
6,2024-06,4614620.94,3694142.98,48.04,920477.96
7,2024-07,4572138.90,3650113.23,45.39,922025.67
8,2024-08,4579761.33,3656963.97,45.46,922797.36
9,2024-09,4551504.34,3639047.70,45.43,912456.64


## ROAS by Category (Marketing ROI)


In [14]:
run_query("""
    SELECT 
        Category,
        ROUND(SUM(Revenue), 2)      AS Total_Revenue,
        ROUND(SUM(Ad_Spend), 2)     AS Total_Ad_Spend,
        ROUND(AVG(ROAS), 2)         AS Avg_ROAS,
        ROUND(AVG(Ad_CTR), 4)       AS Avg_CTR,
        ROUND(AVG(Conversion_Rate), 4) AS Avg_Conversion_Rate
    FROM ecommerce
    GROUP BY Category
    ORDER BY Avg_ROAS DESC
""")

,Category,Total_Revenue,Total_Ad_Spend,Avg_ROAS,Avg_CTR,Avg_Conversion_Rate
0,Books,11195311.88,2109870.58,14.57,0.1055,0.1980
1,Clothing,11034899.26,2123384.33,14.13,0.1043,0.1976
2,Home Appliances,10559207.22,2143318.22,13.76,0.1052,0.2001
3,Toys,11194842.46,2288718.26,13.28,0.1050,0.1955
4,Electronics,11326820.25,2382432.58,12.72,0.1049,0.1996


## Save All Results to Excel

In [15]:
# Save all results to one Excel file
with pd.ExcelWriter(r'C:\Users\bheemanathi\Downloads\ecommerce_sql_results.xlsx') as writer:
    
    run_query("""SELECT ROUND(SUM(Revenue),2) AS Total_Revenue, ROUND(SUM(Profit),2) AS Total_Profit, COUNT(*) AS Total_Orders FROM ecommerce""").to_excel(writer, sheet_name='Overview', index=False)
    
    run_query("""SELECT YearMonth, ROUND(SUM(Revenue),2) AS Revenue, ROUND(SUM(Profit),2) AS Profit FROM ecommerce GROUP BY YearMonth ORDER BY YearMonth""").to_excel(writer, sheet_name='Monthly_Trend', index=False)
    
    run_query("""SELECT Category, ROUND(SUM(Revenue),2) AS Revenue, ROUND(SUM(Profit),2) AS Profit, ROUND(AVG(Profit_Margin_Pct),2) AS Margin FROM ecommerce GROUP BY Category ORDER BY Revenue DESC""").to_excel(writer, sheet_name='By_Category', index=False)
    
    run_query("""SELECT Region, ROUND(SUM(Revenue),2) AS Revenue, ROUND(SUM(Profit),2) AS Profit FROM ecommerce GROUP BY Region ORDER BY Revenue DESC""").to_excel(writer, sheet_name='By_Region', index=False)
    
    run_query("""SELECT Customer_ID, COUNT(*) AS Orders, ROUND(SUM(Revenue),2) AS Revenue FROM ecommerce GROUP BY Customer_ID ORDER BY Revenue DESC LIMIT 100""").to_excel(writer, sheet_name='Top_Customers', index=False)

print("✅ Excel file saved! ecommerce_sql_results.xlsx")

✅ Excel file saved! ecommerce_sql_results.xlsx
